# MNIST-AlexNet-Keras


In [1]:
import numpy as np
import pandas as pd
import keras as keras
import matplotlib.pyplot as plt
%matplotlib inline

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

# Step 1:
# Load & pre-process the image data

X_train: 60000 training images, each with a 28*28 resolution
y_train: array of labels, shows what digit in X_train is.

Keras requires the input to be in a specific format:
(batch, height, width, channels)

Normalize to float between 0 and 1
Original pixel values are between 0 and 255

For y_ label data, we want to convert them into categorical / one-hot variables.
In our case, we have 10 categories, so for example, for our first label in
y_train, the value is 5, then we want to convert it into
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0], with the index of 1 indicating the value of y_train[0]


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255

classes = 10
y_train = np_utils.to_categorical(y_train, classes)
y_test = np_utils.to_categorical(y_test, classes)

# Step 2:
# Build our Neural Network



In [3]:
model = Sequential()

model.add(Conv2D(filters=32, input_shape=(28, 28, 1), kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='valid'))

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='valid'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

# Step 3:
# Prepare & train

We use ImageDataGenerator to augment our input data
which among other benefits, can help reduce over-fitting

In [4]:
gen = ImageDataGenerator(
            rotation_range=8,
            width_shift_range=0.08,
            shear_range=0.3,
            height_shift_range=0.08,
            zoom_range=0.08
            )
test_gen = ImageDataGenerator()

BATCH_SIZE = 64
EPOCHS = 5

train_generator = gen.flow(X_train, y_train, batch_size=BATCH_SIZE)
test_generator = test_gen.flow(X_test, y_test, batch_size=BATCH_SIZE)

import tensorflow as tf

with tf.device("/device:GPU:0"):
    model.fit(
            train_generator,
            steps_per_epoch=60000//BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=test_generator,
            validation_steps=10000//BATCH_SIZE
            )

# Step 4:
# Predict

In [5]:
to_predict = X_test[0]
# reshape from (28, 28, 1) --> (1, 28, 28, 1)
to_predict = np.expand_dims(to_predict, axis=0)

model.predict(to_predict)

# This returns the class with the highest probability
y_predict = np.argmax(model.predict(to_predict), axis=-1)
y_predict[0]

# Step 5:
# Save your model

When you need to load it back, you can just run
and get the same model object as earlier.

In [6]:
from keras.models import load_model

model.save('my_mnist_model.h5')

model = load_model('my_mnist_model.h5')